# Join the respective table to integrate the Customer dimension table

In [0]:
from pyspark.sql.functions import current_timestamp

query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY ci.customer_id) AS customer_key,
    ci.customer_id,
    ci.customer_number,
    ci.first_name,
    ci.last_name,
    la.country,
    ci.marital_status,
    CASE
        WHEN ci.gender <> 'n/a' THEN ci.gender
        ELSE COALESCE(ca.gender, 'n/a')
    END AS gender,
    ca.birth_date AS birthdate,
    ci.created_date AS create_date
FROM dev_project.silver.crm_cust_info ci
LEFT JOIN dev_project.silver.erp_cust_az12 ca
    ON ci.customer_number = ca.customer_number
LEFT JOIN dev_project.silver.erp_loc_a101 la
    ON ci.customer_number = la.customer_number
"""
df = spark.sql(query)

In [0]:
df = df.withColumn("_ingest_time", current_timestamp())

## Display the Results

In [0]:
df.display()

# Writing table into Gold

In [0]:
df.write.mode("overwrite").saveAsTable("dev_project.gold.gold_dim_customers")    

In [0]:
%sql
SELECT * FROM dev_project.gold.gold_dim_customers